In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE159809"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE159809"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE159809.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE159809.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE159809.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Adipose tissue adaptations to an 8-week weight loss intervention in younger and older obese men,"
!Series_summary	"Abdominal subcutaneous adipose tissue transcriptomes were analyzed between 11 young  and 8 elderly obese men during a lifestyle intervention."
!Series_summary	"Lifestyle intervention: Individuals underwent 8-weeks of calorie-restriction of 20% below their daily energy requirement aerobic combined to two sessions of resistance exercise per weeks."
!Series_overall_design	"Two groups,young and elderly; two conditions for each individual, baseline (before lifestyle intervention) and after lifestyle intervention."
Sample Characteristics Dictionary:
{0: ['Sex: mix of male and female']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the background information and sample characteristics
import pandas as pd
import os

# Step 1: Gene expression data availability
# Based on the series title and summary about adipose tissue transcriptomes, this dataset likely contains gene expression data
is_gene_available = True

# Step 2: Variable availability and data type conversion

# Sample Characteristics Dictionary provided shows:
# {0: ['Sex: mix of male and female']}
# Background info mentions two groups (young and elderly) and two conditions (baseline and after intervention)

# 2.1 Data Availability
# For trait (Obesity-related weight loss intervention):
# The background mentions before/after lifestyle intervention - this appears to be the main trait
# However, we don't see it in the sample characteristics dictionary
trait_row = None

# For age:
# The background mentions young vs elderly groups
# But we don't see it in the sample characteristics dictionary
age_row = None

# For gender:
# This is in the sample characteristics dictionary at key 0
gender_row = 0

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert intervention status to binary (0 for baseline, 1 for after intervention)."""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'baseline' in value or 'before' in value or 'pre' in value:
        return 0
    elif 'after' in value or 'post' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to binary (0 for young, 1 for elderly)."""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'young' in value:
        return 0
    elif 'old' in value or 'elder' in value or 'elderly' in value:
        return 1
    else:
        try:
            # Try to extract numeric age if available
            import re
            age_match = re.search(r'(\d+)', value)
            if age_match:
                return float(age_match.group(1))
            else:
                return None
        except:
            return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    # Handle "mix of male and female" case
    elif 'mix' in value:
        return None  # Cannot determine individual gender from this value
    else:
        return None

# Step 3: Save metadata
# Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Skip this step since trait_row is None (clinical trait data not available for our study)
if trait_row is not None:
    # Get the clinical data
    try:
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"), index_col=0)
        
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical data
        print("Selected Clinical Features Preview:")
        print(preview_df(selected_clinical))
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("No clinical data available for extraction and saving.")
else:
    print("No trait data available for extraction and saving.")


No trait data available for extraction and saving.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers with format "A_23_P..." are Agilent microarray probe IDs, not human gene symbols.
# They need to be mapped to official gene symbols for meaningful biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# The gene annotation doesn't contain gene symbol information for the probes
# Let's try a different approach to find the platform annotation

# First, let's check the platform ID from the SOFT file
platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith("^PLATFORM"):
            platform_id = line.strip().split("=")[1].strip()
            break

print(f"Platform ID: {platform_id}")

# Since we identified the platform as GPL13497, we need to find the mapping information
# Let's check if there's a separate platform file in the cohort directory
platform_files = [f for f in os.listdir(in_cohort_dir) if 'GPL13497' in f or 'platform' in f.lower()]
print(f"Platform files found: {platform_files}")

# If we don't find a separate platform file, let's see if the mapping is in the SOFT file but in a different format
# We'll search for sections that might contain probe-to-gene mappings
mapping_sections = []
with gzip.open(soft_file, 'rt') as f:
    current_section = []
    in_mapping_section = False
    
    for line in f:
        line = line.strip()
        if line.startswith("!platform_table_begin") or line.startswith("!Platform_table_begin"):
            in_mapping_section = True
            current_section = []
        elif line.startswith("!platform_table_end") or line.startswith("!Platform_table_end"):
            in_mapping_section = False
            if current_section:
                mapping_sections.append(current_section)
        elif in_mapping_section:
            current_section.append(line)

print(f"Found {len(mapping_sections)} potential mapping sections")

# If we found mapping sections, let's examine them
if mapping_sections:
    for i, section in enumerate(mapping_sections):
        print(f"Section {i+1} first 5 lines:")
        for line in section[:5]:
            print(line)
        print(f"Total lines in section {i+1}: {len(section)}")

# If we still don't have the mapping, let's look for any sections containing probe IDs and gene symbols
# We'll search for probe patterns like A_23_P100001 that we saw in the gene data
import re

probe_pattern = re.compile(r'A_23_P\d+')
genes_and_probes = []

with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if probe_pattern.search(line) and ('gene' in line.lower() or 'symbol' in line.lower()):
            genes_and_probes.append(line.strip())

print(f"Found {len(genes_and_probes)} lines potentially containing probe-gene mappings")
for line in genes_and_probes[:5]:
    print(line)

# Let's try to extract annotation from the gene_data
print("\nLet's examine the gene data itself to see if it contains gene symbols:")
print(gene_data.head())

# As a fallback, we can use platform information from GEO to manually create the mapping
# For Agilent GPL13497, we know it's a common human gene expression microarray
# We'll need to download or access this platform's annotation separately
print("\nFallback approach needed: We need to access platform GPL13497 annotation separately")
print("For now, we'll continue with the probe IDs, and in later steps we can map them to gene symbols")

# Create a simplified annotation dataframe with just the probe IDs from gene_data
id_column = 'ID'  # Using the name of the index in gene_data
probe_ids = pd.DataFrame(gene_data.index).reset_index(drop=True)

print("\nCreated a simple annotation dataframe with probe IDs:")
print(preview_df(probe_ids))


Platform ID: GPL13497
Platform files found: []


Found 1 potential mapping sections
Section 1 first 5 lines:
ID	SPOT_ID	CONTROL_TYPE	REFSEQ	GB_ACC	GENE	GENE_SYMBOL	GENE_NAME	UNIGENE_ID	ENSEMBL_ID	TIGR_ID	ACCESSION_STRING	CHROMOSOMAL_LOCATION	CYTOBAND	DESCRIPTION	GO_ID	SEQUENCE
(+)E1A_r60_1	(+)E1A_r60_1	pos
(+)E1A_r60_3	(+)E1A_r60_3	pos
(+)E1A_r60_a104	(+)E1A_r60_a104	pos
(+)E1A_r60_a107	(+)E1A_r60_a107	pos
Total lines in section 1: 34185


Found 2337 lines potentially containing probe-gene mappings
A_23_P100189	A_23_P100189	FALSE	NM_002761	NM_002761	5619	PRM1	protamine 1	Hs.2909	ENST00000312511		ref|NM_002761|ens|ENST00000312511|gb|Y00443|gb|AY651260	chr16:11374862-11374803	hs|16p13.13	Homo sapiens protamine 1 (PRM1), mRNA [NM_002761]	GO:0000786(nucleosome)|GO:0003677(DNA binding)|GO:0005634(nucleus)|GO:0005654(nucleoplasm)|GO:0005694(chromosome)|GO:0006323(DNA packaging)|GO:0006997(nucleus organization)|GO:0007275(multicellular organismal development)|GO:0007283(spermatogenesis)|GO:0007286(spermatid development)|GO:0030154(cell differentiation)|GO:0030261(chromosome condensation)	GTAGAAGACACTAATTGCACAAAATAGCACATCCACCAAACTCCTGCCTGAGAATGTTAC
A_23_P100392	A_23_P100392	FALSE	NM_007108	NM_007108	6923	TCEB2	transcription elongation factor B (SIII), polypeptide 2 (18kDa, elongin B)	Hs.172772	ENST00000409906		ref|NM_007108|ref|NM_207013|ens|ENST00000409906|ens|ENST00000262306	chr16:2821589-2821530	hs|16p13.3	Homo sapiens transc

### Step 6: Gene Identifier Mapping

In [7]:
# From the gene annotation section in the output, we can see this table structure:
# ID | SPOT_ID | CONTROL_TYPE | REFSEQ | GB_ACC | GENE | GENE_SYMBOL | GENE_NAME | ...

# 1. Extract the probe-to-gene mapping from the platform annotation in the SOFT file
# Based on the section output we saw, we need the ID and GENE_SYMBOL columns
mapping_data = None
with gzip.open(soft_file, 'rt') as f:
    platform_section = False
    header = None
    rows = []
    
    for line in f:
        line = line.strip()
        if line.startswith("!platform_table_begin") or line.startswith("!Platform_table_begin"):
            platform_section = True
            continue
        elif line.startswith("!platform_table_end") or line.startswith("!Platform_table_end"):
            platform_section = False
            continue
        
        if platform_section:
            if header is None:
                header = line.split('\t')
                # Find the index of ID and GENE_SYMBOL columns
                id_idx = header.index('ID')
                symbol_idx = header.index('GENE_SYMBOL')
            else:
                parts = line.split('\t')
                if len(parts) > max(id_idx, symbol_idx):
                    rows.append([parts[id_idx], parts[symbol_idx]])
    
    # Create a DataFrame with the mapping data
    mapping_data = pd.DataFrame(rows, columns=['ID', 'Gene'])

# Clean up the mapping data - remove rows where Gene is empty
mapping_data = mapping_data[mapping_data['Gene'] != '']

# 2. Convert the mapping data to proper format for our gene expression analysis
# Each probe may map to multiple genes (separated by '|')
# We'll split these into separate rows
mapping_data['Gene'] = mapping_data['Gene'].apply(lambda x: [g for g in x.split('|') if g])
mapping_data = mapping_data.explode('Gene')

# Calculate the number of genes per probe for later normalization
probe_to_gene_count = mapping_data.groupby('ID').size().to_dict()
mapping_data['num_genes'] = mapping_data['ID'].map(probe_to_gene_count)

print(f"Mapping data preview (first 5 rows):")
print(preview_df(mapping_data.head()))

# 3. Convert probe-level measurements to gene expression data
# First, ensure gene_data has the ID as regular column, not index
gene_data_reset = gene_data.reset_index()

# Merge with mapping data to assign probe values to genes
merged_data = pd.merge(mapping_data, gene_data_reset, on='ID', how='inner')

# Divide by number of genes per probe (equally distributing signal)
expr_cols = merged_data.columns.drop(['ID', 'Gene', 'num_genes'])
for col in expr_cols:
    merged_data[col] = merged_data[col] / merged_data['num_genes']

# Sum values by gene
gene_data = merged_data.groupby('Gene')[expr_cols].sum()

print("\nConverted gene expression data preview:")
print(preview_df(gene_data.head()))

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Mapping data preview (first 5 rows):
{'ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'Gene': ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'CASC5'], 'num_genes': [1, 1, 1, 1, 1]}

Converted gene expression data preview:
{'GSM4847790': [-5.764699319104582, -0.589637109932678, -3.65899480015227, -0.766907820788277, 0.0031521261184095], 'GSM4847791': [-5.084394785758903, 0.245011071656049, -3.93083850415671, -0.858149965855456, -0.132796092924961], 'GSM4847792': [-5.324133218015119, -0.331121975580139, -3.76917843988412, -1.08089825118244, -0.0221959965676392], 'GSM4847793': [-5.139483793512776, -0.261133235359323, -3.64531518306439, -1.25063532844804, -0.0467902729168449], 'GSM4847794': [-5.2449196345753935, 0.0990713459250179, -3.8062076232237, -0.617110550432826, -0.0167349383904996], 'GSM4847795': [-5.5069333272749015, 0.239607115338824, -3.92822766472244, -0.99562629434523, 0.005496001536267], 'GSM4847796': [-5.25563679057503, -0.461558335863026, -3.7


Gene expression data saved to ../../output/preprocess/Obesity/gene_data/GSE159809.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None (as determined in Step 2), we don't have clinical data to work with
print("No trait data available for this dataset. Proceeding with gene expression data only.")

# Create a minimal DataFrame to use for validation
# Since there's no clinical data, we'll create a minimal DataFrame with the gene expression data
# This allows us to use the validate_and_save_cohort_info function without clinical features
sample_ids = normalized_gene_data.columns.tolist()
linked_data = normalized_gene_data.T  # Transpose so samples are rows

# Add a placeholder Obesity column to allow validation functions to run
linked_data['Obesity'] = 1  # All subjects have obesity (constant trait)
print("Created placeholder data structure for validation purposes")

# Determine trait is biased (no variability - all subjects have obesity)
is_trait_biased = True
print("The trait 'Obesity' is severely biased in this dataset (all subjects have obesity).")

# No demographic features to evaluate for bias since they aren't available
print("No demographic features (Age, Gender) available to evaluate for bias.")

# Conduct final validation
note = "This dataset contains gene expression data from obese individuals only, making it unsuitable for obesity vs. non-obesity association studies. It appears to be a study of pre/post intervention in obese individuals, but that trait information is not clearly encoded in the available data."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,  # False as determined in Step 2
    is_biased=is_trait_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability status: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for obesity trait-gene association studies.")

Normalized gene data saved to ../../output/preprocess/Obesity/gene_data/GSE159809.csv
No trait data available for this dataset. Proceeding with gene expression data only.
Created placeholder data structure for validation purposes
The trait 'Obesity' is severely biased in this dataset (all subjects have obesity).
No demographic features (Age, Gender) available to evaluate for bias.
Dataset usability status: False
Dataset is not usable for obesity trait-gene association studies.
